# Week 3 Assignment: Segmentation and Clustering of Neighborhoods in Toronto

## Task 3: Segmenting and Clustering Neighborhoods of Boroughs with names including 'Toronto'

### Importing required libraries

In [65]:
# Standard libraries
import pandas as pd
import numpy as np

# Library needed to read websites
from urllib.request import urlopen

# Libraries to extract and process geodata and JSON files
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe
import geocoder

# Libraries to create and manipulate maps
import folium
## Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Reading data from Wikipedia (see Task 1 for details)

In [5]:
raw_data = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = raw_data[0]
df=df[df.Borough!='Not assigned']
df.reset_index(drop=True,inplace=True)
print('There are {} assigned postal codes distributed throughout the {} boroughs of Toronto.'.format(len(df),str(len(df['Borough'].unique()))))

There are 103 assigned postal codes distributed throughout the 10 boroughs of Toronto.


### Importing geodata of neighborhodds (see Task 2 for details)

In [10]:
bing_API = '' # add your bing API key if you have one
df_coords = pd.DataFrame({'Postal Code': [], 'latitude' : [], 'longitude' : []})
for pc in df['Postal Code']:
    g = geocoder.bing(pc+', Toronto, Ontario',key=bing_API)
    c = pd.DataFrame({'Postal Code': [pc], 'latitude' : [g.lat], 'longitude' : [g.lng]})
    df_coords = df_coords.append(c)
merge_df = pd.merge(df,df_coords,on='Postal Code')
merge_df.head()


,Postal Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.751881,-79.330360
1,M4A,North York,Victoria Village,43.730419,-79.312820
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655140,-79.362648
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723209,-79.451408
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.664490,-79.393021


### Preparing a dataframe for clustering
From the merged data frame, data for all boroughs with names including Toronto explicitly will be extracted to be subsequently investigated.

In [28]:
tor_data = merge_df[merge_df.Borough.str.contains('Toronto')] # Creates a data frame with data only from Downtown
tor_data.reset_index(drop=True,inplace=True) # Re-sets the indices from the original data frame to 0 through n
tor_data.head()

,Postal Code,Borough,Neighborhood,latitude,longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655140,-79.362648
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.664490,-79.393021
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657360,-79.378181
3,M5C,Downtown Toronto,St. James Town,43.651428,-79.375572
4,M4E,East Toronto,The Beaches,43.677029,-79.295418


In [29]:
print('{} neighborhoods in {} boroughs will be analysed.'.format(len(tor_data),str(len(tor_data['Borough'].unique()))))

39 neighborhoods in 4 boroughs will be analysed.


Let's have a look at how these neighborhoods are distributed geographically. To this end a map of Toronto with markers for the neighborhoods will be created. First, the coordinates of Toronto will be extracted.

In [31]:
t = geocoder.bing('Toronto, Ontario',key=bing_API)
print('The geograpical coordinates of Toronto are {}, {}.'.format(t.lat,t.lng))

The geograpical coordinates of Toronto are 43.64868927001953, -79.38543701171875.


Now, a map will be created.

In [44]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t.lat, t.lng], zoom_start=12)
# add markers to map
for lat, lng, label in zip(tor_data['latitude'], tor_data['longitude'], tor_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

### Clustering of the neighborhoods
The following block of code will follow the lab on the clustering of neighborhoods in Manhatten.
#### Scraping data for each neighborhood using Foursquare

In [38]:
#Input your Foursquare credentials!
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200517' # Foursquare API version
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VP5K2R0IUFG0PNIMBLUUWCTYLIN0VI12LYNPFIE05JSVQFTB
CLIENT_SECRET:R3MJF1YNU32BE1IMON1H0DCWI3K0QKKYLHXV0WV3ZVLS21GP


The following code defines a function that looks up venues in a defined radius around a coordinate. Since the neighborhoods are quite small, the radius as set to 500 m.

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The function is now called for all defined neighborhoods.

In [52]:
toronto_venues = getNearbyVenues(names=tor_data['Neighborhood'],
                                   latitudes=tor_data['latitude'],
                                   longitudes=tor_data['longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [53]:
print('{} venues were found in the analysed {} neighborhoods.'.format(len(toronto_venues),len(tor_data)))
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

1715 venues were found in the analysed 39 neighborhoods.
There are 219 unique categories.


Next, we are going to analyse the neighborhoods by counting the occurences of each type of venue in a neighborhood and calculating their frequency.

In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

Following the lab, I will now create a dataframe with the top 10 venues of each neighborhood. First, the function is defined, then it is called.

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [160]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Seafood Restaurant,Cocktail Bar,Breakfast Spot,Café,Lounge,Bakery,Beer Bar,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Lounge,Gift Shop,Sandwich Place,Japanese Restaurant,Breakfast Spot,Nightclub
2,Business reply mail Processing Centre,Coffee Shop,Hotel,Restaurant,Café,Japanese Restaurant,Concert Hall,Gym,Bar,Steakhouse,Italian Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Café,Restaurant,Bar,Gym / Fitness Center,Speakeasy,Sandwich Place,Electronics Store,Bakery
4,Central Bay Street,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Bubble Tea Shop,Plaza,Sandwich Place,Hotel,Restaurant,Café


### Clustering the neighborhoods
Now that the venues in the neighborhoods have been identified and their frequency has been calculated, we can start clustering them using kmeans.

In [165]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 4], dtype=int32)

In [166]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [167]:
toronto_merged = tor_data.drop([19]) # this neighborhood returns a NaN for the clustering, leading to float labels.

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head(2)

,Postal Code,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.362648,1,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Gym / Fitness Center,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.393021,1,Coffee Shop,Café,Park,Museum,Chinese Restaurant,Sandwich Place,Salon / Barbershop,Restaurant,Clothing Store,Pub


Now, the data for the neighborhoods is clustered and ready to be put on a map, which will be done in the next step.

In [169]:
# create map
map_clusters = folium.Map(location=[t.lat, t.lng], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

After trying mulitple combinations of number of clusters ranging from 3 to 10 and the number of considered venue types from top3 to top20, I came to the conclusion, that the analysed neighborhoods are all very similar. Only a few are parts of other "clusters" containing one or two neighborhoods. 
Let us have a look at how they are distinct from the typical neighborhood in Toronto.

### Identifying cluster characteristics
To identify the characteristics that distinguish the clusters, we will have a look at the top venues in each cluster.

In [171]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,"Summerhill West, Rathnelly, South Hill, Forest...",0,Coffee Shop,Light Rail Station,Liquor Store,Supermarket,Women's Store,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [173]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]].head()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Gym / Fitness Center,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Café,Park,Museum,Chinese Restaurant,Sandwich Place,Salon / Barbershop,Restaurant,Clothing Store,Pub
2,"Garden District, Ryerson",1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Restaurant,Café,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Bookstore
3,St. James Town,1,Coffee Shop,Café,Seafood Restaurant,Cocktail Bar,American Restaurant,Gastropub,Italian Restaurant,Bakery,Creperie,Cosmetics Shop
4,The Beaches,1,Health Food Store,Pub,College Gym,Trail,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [174]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,2,Park,Women's Store,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
21,Forest Hill North & West,2,Park,Women's Store,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
22,"High Park, The Junction South",2,Park,Women's Store,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


In [175]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North Toronto West,3,Playground,Gym Pool,Park,Garden,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
33,Rosedale,3,Playground,Park,Building,Tennis Court,Creperie,Cuban Restaurant,Fast Food Restaurant,Cosmetics Shop,Farmers Market,Farm


In [176]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Cluster 2 is very large. Only the first five neighborhoods are shown. The fifth cluster is completely empty.
The multiple neighborhoods that are part of the only entry in cluster 1 differ from the rest, by the Railstations.
Cluster 2 is dominated by Cafes and Coffee Shops, Pubs and Restaurants. 
The neighborhoods in cluster 3 are almost identical. They are the light blue entries on the map. It is directly visible that they are not directly connected, which is surprising. 
Cluster 4 is very similar to cluster 3, but a bit more diverse. The most common venue here are playgrounds, making these areas more suitable for families with children.
Cluster 5 as mentioned earlier is empty.